In [3]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

In [4]:
# read data
df = pd.read_csv('../data/feature_selection_negative.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [7]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=1, 
    epochs=100, 
    batch_size=1000
)

In [8]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [9]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [10]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
248/248 [==============================] - 37s 150ms/step - loss: 2.1224 - acc: 0.2581
Epoch 2/100
248/248 [==============================] - 0s 555us/step - loss: 2.1022 - acc: 0.2661
Epoch 3/100
248/248 [==============================] - 0s 576us/step - loss: 1.6668 - acc: 0.3387
Epoch 4/100
248/248 [==============================] - 0s 574us/step - loss: 1.8977 - acc: 0.3024
Epoch 5/100
248/248 [==============================] - 0s 450us/step - loss: 1.9159 - acc: 0.3105
Epoch 6/100
248/248 [==============================] - 0s 483us/step - loss: 1.6731 - acc: 0.3548
Epoch 7/100
248/248 [==============================] - 0s 59us/step - loss: 1.7583 - acc: 0.3750
Epoch 8/100
248/248 [==============================] - 0s 894us/step - loss: 1.6078 - acc: 0.3831
Epoch 9/100
248/248 [==============================] - 0s 598us/step - loss: 1.6607 - acc: 0.3669
Epoch 10/100
248/248 [==============================] - 0s 559us/step - loss: 1.4382 - acc: 0.4194
Epoch 11/100
248/24

Epoch 85/100
248/248 [==============================] - 0s 669us/step - loss: 0.9805 - acc: 0.5968
Epoch 86/100
248/248 [==============================] - 0s 636us/step - loss: 0.9382 - acc: 0.5726
Epoch 87/100
248/248 [==============================] - 0s 526us/step - loss: 0.9671 - acc: 0.5565
Epoch 88/100
248/248 [==============================] - 0s 283us/step - loss: 1.0130 - acc: 0.5806
Epoch 89/100
248/248 [==============================] - 0s 184us/step - loss: 0.9715 - acc: 0.5605
Epoch 90/100
248/248 [==============================] - 0s 849us/step - loss: 0.9974 - acc: 0.5806
Epoch 91/100
248/248 [==============================] - 0s 510us/step - loss: 0.9628 - acc: 0.6250
Epoch 92/100
248/248 [==============================] - 0s 713us/step - loss: 0.9560 - acc: 0.5887
Epoch 93/100
248/248 [==============================] - 0s 303us/step - loss: 0.9534 - acc: 0.5887
Epoch 94/100
248/248 [==============================] - 0s 625us/step - loss: 0.9886 - acc: 0.5847
Epoch 95/1

Epoch 67/100
249/249 [==============================] - 0s 490us/step - loss: 1.0616 - acc: 0.5301
Epoch 68/100
249/249 [==============================] - 0s 670us/step - loss: 1.0404 - acc: 0.5904
Epoch 69/100
249/249 [==============================] - 0s 652us/step - loss: 1.1207 - acc: 0.5020
Epoch 70/100
249/249 [==============================] - 0s 653us/step - loss: 1.1440 - acc: 0.5141
Epoch 71/100
249/249 [==============================] - 0s 673us/step - loss: 1.0833 - acc: 0.5382
Epoch 72/100
249/249 [==============================] - 0s 649us/step - loss: 1.1114 - acc: 0.4940
Epoch 73/100
249/249 [==============================] - 0s 718us/step - loss: 1.0458 - acc: 0.5944
Epoch 74/100
249/249 [==============================] - 0s 124us/step - loss: 1.0946 - acc: 0.5823
Epoch 75/100
249/249 [==============================] - 0s 314us/step - loss: 1.0707 - acc: 0.5622
Epoch 76/100
249/249 [==============================] - 0s 157us/step - loss: 1.0233 - acc: 0.5622
Epoch 77/1

Epoch 49/100
249/249 [==============================] - 0s 560us/step - loss: 1.0607 - acc: 0.5341
Epoch 50/100
249/249 [==============================] - 0s 520us/step - loss: 1.1958 - acc: 0.5100
Epoch 51/100
249/249 [==============================] - 0s 531us/step - loss: 1.1779 - acc: 0.4859
Epoch 52/100
249/249 [==============================] - 0s 598us/step - loss: 0.9923 - acc: 0.5462
Epoch 53/100
249/249 [==============================] - 0s 287us/step - loss: 1.1104 - acc: 0.5141
Epoch 54/100
249/249 [==============================] - 0s 487us/step - loss: 1.1863 - acc: 0.5020
Epoch 55/100
249/249 [==============================] - 0s 562us/step - loss: 1.1045 - acc: 0.5060
Epoch 56/100
249/249 [==============================] - 0s 540us/step - loss: 1.1122 - acc: 0.5502
Epoch 57/100
249/249 [==============================] - 0s 63us/step - loss: 1.1191 - acc: 0.5060
Epoch 58/100
249/249 [==============================] - 0s 524us/step - loss: 1.2236 - acc: 0.4538
Epoch 59/10

Epoch 31/100
248/248 [==============================] - 0s 379us/step - loss: 0.5217 - acc: 0.8024
Epoch 32/100
248/248 [==============================] - 0s 396us/step - loss: 0.5470 - acc: 0.7823
Epoch 33/100
248/248 [==============================] - 0s 481us/step - loss: 0.6131 - acc: 0.7460
Epoch 34/100
248/248 [==============================] - 0s 514us/step - loss: 0.7184 - acc: 0.7460
Epoch 35/100
248/248 [==============================] - 0s 591us/step - loss: 0.4984 - acc: 0.8065
Epoch 36/100
248/248 [==============================] - 0s 545us/step - loss: 0.5435 - acc: 0.7944
Epoch 37/100
248/248 [==============================] - 0s 664us/step - loss: 0.5079 - acc: 0.7661
Epoch 38/100
248/248 [==============================] - 0s 699us/step - loss: 0.5740 - acc: 0.8024
Epoch 39/100
248/248 [==============================] - 0s 604us/step - loss: 0.5233 - acc: 0.7742
Epoch 40/100
248/248 [==============================] - 0s 243us/step - loss: 0.5120 - acc: 0.8065
Epoch 41/1

...

Epoch 10/100
248/248 [==============================] - 0s 104us/step - loss: 0.7404 - acc: 0.6855
Epoch 11/100
248/248 [==============================] - 0s 225us/step - loss: 0.8932 - acc: 0.6653
Epoch 12/100
248/248 [==============================] - 0s 297us/step - loss: 1.0210 - acc: 0.6290
Epoch 13/100
248/248 [==============================] - 0s 231us/step - loss: 0.8657 - acc: 0.6774
Epoch 14/100
248/248 [==============================] - 0s 228us/step - loss: 0.6671 - acc: 0.7298
Epoch 15/100
248/248 [==============================] - 0s 462us/step - loss: 0.7173 - acc: 0.6895
Epoch 16/100
248/248 [==============================] - 0s 783us/step - loss: 0.7619 - acc: 0.6935
Epoch 17/100
248/248 [==============================] - 0s 647us/step - loss: 0.6525 - acc: 0.7661
Epoch 18/100
248/248 [==============================] - 0s 491us/step - loss: 0.6483 - acc: 0.7097
Epoch 19/100
248/248 [==============================] - 0s 804us/step - loss: 0.6150 - acc: 0.7702
Epoch 20/1

Epoch 93/100
248/248 [==============================] - 0s 412us/step - loss: 0.1322 - acc: 0.9556
Epoch 94/100
248/248 [==============================] - 0s 412us/step - loss: 0.1240 - acc: 0.9597
Epoch 95/100
248/248 [==============================] - 0s 381us/step - loss: 0.1369 - acc: 0.9435
Epoch 96/100
248/248 [==============================] - 0s 379us/step - loss: 0.1457 - acc: 0.9435
Epoch 97/100
248/248 [==============================] - 0s 495us/step - loss: 0.1738 - acc: 0.9355
Epoch 98/100
248/248 [==============================] - 0s 100us/step - loss: 0.1895 - acc: 0.9073
Epoch 99/100
248/248 [==============================] - 0s 158us/step - loss: 0.1860 - acc: 0.9355
Epoch 100/100
248/248 [==============================] - 0s 607us/step
Epoch 1/100
249/249 [==============================] - 104s 416ms/step - loss: 2.1753 - acc: 0.2289
Epoch 2/100
249/249 [==============================] - 0s 670us/step - loss: 2.0544 - acc: 0.4578
Epoch 3/100
249/249 [=================

Epoch 75/100
249/249 [==============================] - 0s 195us/step - loss: 0.2300 - acc: 0.8916
Epoch 76/100
249/249 [==============================] - 0s 137us/step - loss: 0.2028 - acc: 0.9036
Epoch 77/100
249/249 [==============================] - 0s 940us/step - loss: 0.1887 - acc: 0.9317
Epoch 78/100
249/249 [==============================] - 0s 414us/step - loss: 0.1709 - acc: 0.9357
Epoch 79/100
249/249 [==============================] - 0s 806us/step - loss: 0.2194 - acc: 0.9197
Epoch 80/100
249/249 [==============================] - 0s 623us/step - loss: 0.1857 - acc: 0.9237
Epoch 81/100
249/249 [==============================] - 0s 559us/step - loss: 0.1765 - acc: 0.9237
Epoch 82/100
249/249 [==============================] - 0s 889us/step - loss: 0.1888 - acc: 0.9116
Epoch 83/100
249/249 [==============================] - 0s 616us/step - loss: 0.2349 - acc: 0.8996
Epoch 84/100
249/249 [==============================] - 0s 331us/step - loss: 0.1612 - acc: 0.9398
Epoch 85/1

Epoch 57/100
249/249 [==============================] - 0s 547us/step - loss: 0.3206 - acc: 0.8876
Epoch 58/100
249/249 [==============================] - 0s 552us/step - loss: 0.3653 - acc: 0.8434
Epoch 59/100
249/249 [==============================] - 0s 509us/step - loss: 0.3521 - acc: 0.8353
Epoch 60/100
249/249 [==============================] - 0s 614us/step - loss: 0.4261 - acc: 0.8394
Epoch 61/100
249/249 [==============================] - 0s 589us/step - loss: 0.3173 - acc: 0.8635
Epoch 62/100
249/249 [==============================] - 0s 390us/step - loss: 0.2658 - acc: 0.9036
Epoch 63/100
249/249 [==============================] - 0s 517us/step - loss: 0.2677 - acc: 0.8755
Epoch 64/100
249/249 [==============================] - 0s 467us/step - loss: 0.2709 - acc: 0.8956
Epoch 65/100
249/249 [==============================] - 0s 756us/step - loss: 0.2393 - acc: 0.9116
Epoch 66/100
249/249 [==============================] - 0s 477us/step - loss: 0.2546 - acc: 0.8755
Epoch 67/1

Epoch 39/100
373/373 [==============================] - 0s 508us/step - loss: 0.5422 - acc: 0.7802
Epoch 40/100
373/373 [==============================] - 0s 379us/step - loss: 0.5771 - acc: 0.7453
Epoch 41/100
373/373 [==============================] - 0s 478us/step - loss: 0.5572 - acc: 0.7587
Epoch 42/100
373/373 [==============================] - 0s 385us/step - loss: 0.6094 - acc: 0.7507
Epoch 43/100
373/373 [==============================] - 0s 599us/step - loss: 0.5845 - acc: 0.7292
Epoch 44/100
373/373 [==============================] - 0s 585us/step - loss: 0.5281 - acc: 0.7694
Epoch 45/100
373/373 [==============================] - 0s 470us/step - loss: 0.5407 - acc: 0.7802
Epoch 46/100
373/373 [==============================] - 0s 551us/step - loss: 0.5522 - acc: 0.7587
Epoch 47/100
373/373 [==============================] - 0s 442us/step - loss: 0.5032 - acc: 0.7962
Epoch 48/100
373/373 [==============================] - 0s 436us/step - loss: 0.5049 - acc: 0.7855
Epoch 49/1

In [11]:
t = grid_result.best_estimator_

In [12]:
t.score(X_test, y_test)

94/94 [==============================] - 26s 275ms/step


0.7021276354789734

In [13]:
t.model.save('../model/Keras_best_params_neg_fs.h5')